# Multi-Account EC2 Cost Optimization Report

**Comprehensive analysis of EC2 spending across multiple AWS accounts**

This notebook provides actionable insights for Engineering and DevOps teams to optimize AWS EC2 costs across your organization.

---

## Configuration

In [ ]:
# === CONFIGURATION ===
import glob
import os

# Auto-detect latest multi-account cost report
report_files = glob.glob("multi_account_cost_report_*.json") + glob.glob("data/multi_account_cost_report_*.json")
if report_files:
    REPORT_JSON_PATH = max(report_files, key=os.path.getmtime)
else:
    raise FileNotFoundError("No multi-account cost report found. Run: uv run python multi_account_analyzer.py")

DEFAULT_REGION = "eu-west-2"  # Default AWS region for commands
# === END CONFIGURATION ===

print(f"Using report: {REPORT_JSON_PATH}")

## Setup & Imports

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import warnings
from datetime import datetime

# Import utility functions
from utils.report_utils import (
    # Multi-account functions
    load_multi_account_report, get_account_list, get_account_data, get_aggregated_data,
    parse_account_costs,
    create_account_cost_bar_chart, create_account_cost_pie_chart,
    create_multi_account_monthly_trend, create_multi_account_kpi_cards,
    create_multi_account_savings_waterfall,
    # Single-account functions (for drill-down)
    parse_monthly_costs, parse_instance_type_costs, parse_region_costs, parse_tag_costs,
    parse_cloudwatch_instances, parse_ebs_analysis, parse_spot_analysis, parse_ri_sp_analysis,
    parse_fiscal_year_forecast, parse_service_costs,
    create_monthly_trend_chart, create_horizontal_bar_chart, create_pie_chart,
    create_gauge_chart, create_waterfall_chart, create_fiscal_year_forecast_chart,
    create_fiscal_year_summary_kpis,
    generate_stop_instance_command, generate_batch_stop_script,
    format_currency, format_percentage
)

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)

print("Libraries imported successfully")

## Load Data

In [ ]:
# Load multi-account report
try:
    data = load_multi_account_report(REPORT_JSON_PATH)
    print(f"Loaded multi-account cost report: {REPORT_JSON_PATH}")
    print(f"Generated: {data['generated_at']}")
    print(f"Accounts analyzed: {data['accounts_analyzed']}")
    if data.get('accounts_failed', 0) > 0:
        print(f"Accounts failed: {data['accounts_failed']}")
    print("\nAccounts included:")
    for acc in get_account_list(data):
        print(f"  - {acc['account_name']}: ${acc['total_cost']:,.2f}")
except Exception as e:
    print(f"Error loading report: {e}")
    raise

---
# Section 1: Multi-Account Executive Summary

High-level overview of costs across all AWS accounts.

In [ ]:
# Get aggregated data
aggregated = get_aggregated_data(data)

total_ec2_cost = aggregated.get('total_ec2_cost', 0)
total_aws_cost = aggregated.get('total_aws_cost', total_ec2_cost)
total_optimization = aggregated.get('total_optimization_potential', 0)
accounts_count = data.get('accounts_analyzed', 0)

print("\n" + "="*80)
print("MULTI-ACCOUNT EXECUTIVE SUMMARY")
print("="*80)
print(f"\nAccounts Analyzed: {accounts_count}")
print(f"Total EC2 Cost (3-month): ${total_ec2_cost:,.2f}")
print(f"Total AWS Cost (3-month): ${total_aws_cost:,.2f}")
print(f"Total Optimization Potential: ${total_optimization:,.2f}/month (${total_optimization*12:,.2f}/year)")
print("="*80)

In [ ]:
# Multi-Account KPI Cards
fig = create_multi_account_kpi_cards(data)
fig.show()

## 1.1 Cost by Account

In [ ]:
# Cost by Account - Bar Chart
fig = create_account_cost_bar_chart(data, "EC2 Cost by Account (3-Month)")
fig.show()

In [ ]:
# Cost by Account - Pie Chart
fig = create_account_cost_pie_chart(data, "Cost Distribution by Account")
fig.show()

## 1.2 Monthly Cost Trend by Account

In [ ]:
# Stacked Area Chart - Monthly Costs by Account
fig = create_multi_account_monthly_trend(data, "Monthly Cost Trend by Account")
fig.show()

## 1.3 Aggregated Fiscal Year Forecast

In [ ]:
# Aggregated Fiscal Year Forecast
fy_data = aggregated.get('fiscal_year_forecast', {})

if fy_data:
    total_actual = fy_data.get('total_actual', 0)
    total_forecast = fy_data.get('total_forecast', 0)
    total_projected = fy_data.get('total_projected', 0)
    
    print("\n" + "="*80)
    print("AGGREGATED FISCAL YEAR FORECAST")
    print("="*80)
    print(f"\nActual Spend (YTD):      ${total_actual:>15,.2f}")
    print(f"Forecasted Remaining:    ${total_forecast:>15,.2f}")
    print(f"{'-'*50}")
    print(f"Projected FY Total:      ${total_projected:>15,.2f}")
    print("="*80)
else:
    print("No aggregated fiscal year forecast data available.")

---
# Section 2: Aggregated Cost Breakdown

Cost breakdown across all accounts.

## 2.1 Top Instance Types by Cost (All Accounts)

In [ ]:
# Aggregated instance type costs
instance_type_df = parse_instance_type_costs(aggregated, top_n=15)

if not instance_type_df.empty:
    fig = create_horizontal_bar_chart(instance_type_df, 'cost', 'instance_type',
                                       'Top 15 Instance Types by Cost (All Accounts)', 'Cost ($)')
    fig.show()
    
    print("\nTop 5 Most Expensive Instance Types:")
    for idx, row in instance_type_df.head(5).iterrows():
        print(f"  {row['instance_type']:20s} ${row['cost']:>12,.2f} ({row['percentage']:.1f}%)")
else:
    print("No instance type data available.")

## 2.2 Cost by Region (All Accounts)

In [ ]:
# Aggregated region costs
region_df = parse_region_costs(aggregated)

if not region_df.empty:
    fig = create_pie_chart(region_df, 'cost', 'region', 'Cost by Region (All Accounts)')
    fig.show()
    
    print("\nCost by Region:")
    for idx, row in region_df.iterrows():
        print(f"  {row['region']:20s} ${row['cost']:>12,.2f} ({row['percentage']:.1f}%)")
else:
    print("No region data available.")

---
# Section 3: Optimization Potential by Account

Savings opportunities identified across all accounts.

In [ ]:
# Multi-account savings waterfall
total_optimization = aggregated.get('total_optimization_potential', 0)

print("\n" + "="*80)
print("OPTIMIZATION POTENTIAL BY ACCOUNT")
print("="*80)
print(f"\nTotal Monthly Optimization Potential: ${total_optimization:,.2f}")
print(f"Total Annual Optimization Potential: ${total_optimization * 12:,.2f}")
print("="*80)

fig = create_multi_account_savings_waterfall(data, "Monthly Optimization Potential by Account")
fig.show()

In [ ]:
# Summary table of savings by account
account_reports = data.get('account_reports', {})
savings_summary = []

for account_id, account_data in account_reports.items():
    account_name = account_data.get('account_name', account_id)
    total_cost = account_data.get('total_cost', 0)
    action_plan = account_data.get('action_plan', {})
    total_savings = action_plan.get('total_savings', 0)
    
    savings_summary.append({
        'Account': account_name,
        'Total Cost (3-mo)': total_cost,
        'Monthly Savings': total_savings,
        'Annual Savings': total_savings * 12,
        'Savings %': (total_savings / (total_cost / 3) * 100) if total_cost > 0 else 0
    })

savings_df = pd.DataFrame(savings_summary)
savings_df = savings_df.sort_values('Monthly Savings', ascending=False)

print("\nSavings Potential by Account:\n")
display_df = savings_df.copy()
display_df['Total Cost (3-mo)'] = display_df['Total Cost (3-mo)'].apply(lambda x: f"${x:,.2f}")
display_df['Monthly Savings'] = display_df['Monthly Savings'].apply(lambda x: f"${x:,.2f}")
display_df['Annual Savings'] = display_df['Annual Savings'].apply(lambda x: f"${x:,.2f}")
display_df['Savings %'] = display_df['Savings %'].apply(lambda x: f"{x:.1f}%")
display(display_df)

---
# Section 4: Per-Account Drill-Down

Select an account to view detailed analysis including resource utilization, storage optimization, and specific recommendations.

In [ ]:
# Account selector dropdown
account_list = get_account_list(data)
account_options = [(f"{acc['account_name']} (${acc['total_cost']:,.0f})", acc['account_id']) for acc in account_list]

account_dropdown = widgets.Dropdown(
    options=account_options,
    description='Select Account:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

output = widgets.Output()

def display_account_details(account_id):
    """Display detailed analysis for selected account."""
    account_data = get_account_data(data, account_id)
    account_name = account_data.get('account_name', account_id)
    
    print("\n" + "="*80)
    print(f"DETAILED ANALYSIS: {account_name}")
    print(f"Account ID: {account_id}")
    print("="*80)
    
    # Summary
    total_cost = account_data.get('total_cost', 0)
    action_plan = account_data.get('action_plan', {})
    total_savings = action_plan.get('total_savings', 0)
    
    print(f"\nTotal EC2 Cost (3-month): ${total_cost:,.2f}")
    print(f"Optimization Potential: ${total_savings:,.2f}/month")
    
    # Monthly trend
    print("\n--- Monthly Cost Trend ---")
    monthly_df = parse_monthly_costs(account_data)
    if not monthly_df.empty:
        fig = create_monthly_trend_chart(monthly_df)
        fig.update_layout(title=f"Monthly Cost Trend - {account_name}")
        fig.show()
    
    # Instance types
    print("\n--- Top Instance Types ---")
    instance_df = parse_instance_type_costs(account_data, top_n=10)
    if not instance_df.empty:
        fig = create_horizontal_bar_chart(instance_df, 'cost', 'instance_type',
                                           f'Top 10 Instance Types - {account_name}', 'Cost ($)')
        fig.show()
    
    # Region breakdown
    print("\n--- Cost by Region ---")
    region_df = parse_region_costs(account_data)
    if not region_df.empty:
        for idx, row in region_df.iterrows():
            print(f"  {row['region']:20s} ${row['cost']:>12,.2f} ({row['percentage']:.1f}%)")
    
    # Cloudwatch analysis (idle/underutilized)
    cw_analysis = account_data.get('cloudwatch_analysis', {})
    idle_instances = cw_analysis.get('idle', [])
    underutil_instances = cw_analysis.get('underutilized', [])
    
    if idle_instances:
        print(f"\n--- Idle Instances ({len(idle_instances)} found) ---")
        idle_df = pd.DataFrame(idle_instances)
        if 'InstanceId' in idle_df.columns:
            display(idle_df[['InstanceId', 'InstanceType', 'AvgCPU']].head(10))
    
    if underutil_instances:
        print(f"\n--- Underutilized Instances ({len(underutil_instances)} found) ---")
        underutil_df = pd.DataFrame(underutil_instances)
        if 'InstanceId' in underutil_df.columns:
            display(underutil_df[['InstanceId', 'InstanceType', 'AvgCPU']].head(10))
    
    # EBS Analysis
    ebs_data = account_data.get('ebs_analysis', {})
    if ebs_data:
        print("\n--- Storage Optimization ---")
        print(f"  Unattached volumes: {ebs_data.get('unattached_count', 0)}")
        print(f"  Unattached savings: ${ebs_data.get('unattached_savings', 0):,.2f}/month")
        print(f"  GP2->GP3 migration savings: ${ebs_data.get('gp2_migration_savings', 0):,.2f}/month")
    
    # Spot opportunities
    spot_data = account_data.get('spot_analysis', {})
    if spot_data:
        print("\n--- Spot Instance Opportunities ---")
        print(f"  Eligible instances: {spot_data.get('eligible_count', 0)}")
        print(f"  Potential savings: ${spot_data.get('savings', 0):,.2f}/month")
    
    # RI/SP Analysis
    ri_sp = account_data.get('ri_sp_analysis', {})
    if ri_sp:
        print("\n--- Savings Plans / Reserved Instances ---")
        print(f"  Current coverage: {ri_sp.get('coverage', 0):.1f}%")
        print(f"  Potential savings: ${ri_sp.get('potential_savings', 0):,.2f}/month")
    
    # Action Plan
    if action_plan:
        print("\n--- Action Plan Summary ---")
        print(f"  Total monthly savings: ${total_savings:,.2f}")
        
        for priority in ['immediate', 'high_priority', 'medium_priority']:
            actions = action_plan.get(priority, [])
            if actions:
                print(f"\n  {priority.replace('_', ' ').title()}:")
                for action in actions[:3]:  # Show top 3 per priority
                    if isinstance(action, dict):
                        print(f"    - {action.get('action', 'N/A')}: ${action.get('savings', 0):,.2f}/mo")

def on_account_change(change):
    with output:
        clear_output(wait=True)
        display_account_details(change['new'])

account_dropdown.observe(on_account_change, names='value')

print("Select an account to view detailed analysis:")
display(account_dropdown)
display(output)

# Trigger initial display
if account_options:
    with output:
        display_account_details(account_options[0][1])

---
# Section 5: Key Recommendations Summary

Top actionable items across all accounts.

In [ ]:
# Aggregate recommendations from all accounts
all_recommendations = []

for account_id, account_data in account_reports.items():
    account_name = account_data.get('account_name', account_id)
    action_plan = account_data.get('action_plan', {})
    
    for priority in ['immediate', 'high_priority', 'medium_priority']:
        actions = action_plan.get(priority, [])
        for action in actions:
            if isinstance(action, dict):
                all_recommendations.append({
                    'Account': account_name,
                    'Priority': priority.replace('_', ' ').title(),
                    'Action': action.get('action', 'N/A'),
                    'Monthly Savings': action.get('savings', 0),
                    'Category': action.get('category', 'N/A')
                })

if all_recommendations:
    rec_df = pd.DataFrame(all_recommendations)
    rec_df = rec_df.sort_values('Monthly Savings', ascending=False)
    
    print("\n" + "="*80)
    print("TOP 15 RECOMMENDATIONS ACROSS ALL ACCOUNTS (Ranked by Savings)")
    print("="*80 + "\n")
    
    for idx, row in rec_df.head(15).iterrows():
        print(f"{idx+1}. [{row['Priority']}] {row['Action']}")
        print(f"   Account: {row['Account']}")
        print(f"   Savings: ${row['Monthly Savings']:,.2f}/month")
        print()
else:
    print("No recommendations available.")

---
# Export Options

Export data for implementation and tracking.

In [ ]:
# Export account cost summary to CSV
account_summary = []
for acc in get_account_list(data):
    account_data = get_account_data(data, acc['account_id'])
    action_plan = account_data.get('action_plan', {})
    account_summary.append({
        'Account ID': acc['account_id'],
        'Account Name': acc['account_name'],
        'Total Cost (3-mo)': acc['total_cost'],
        'Monthly Savings Potential': action_plan.get('total_savings', 0)
    })

summary_df = pd.DataFrame(account_summary)
csv_filename = f"multi_account_summary_{datetime.now().strftime('%Y%m%d')}.csv"
summary_df.to_csv(csv_filename, index=False)
print(f"Account summary exported to: {csv_filename}")

# Export all recommendations
if all_recommendations:
    rec_df = pd.DataFrame(all_recommendations)
    rec_filename = f"multi_account_recommendations_{datetime.now().strftime('%Y%m%d')}.csv"
    rec_df.to_csv(rec_filename, index=False)
    print(f"Recommendations exported to: {rec_filename}")

print("\nTo export this notebook as HTML:")
print("   jupyter nbconvert --to html ec2_cost_optimization_report.ipynb")

---
# Report Complete

**Next Steps:**
1. Review the optimization potential by account
2. Use the drill-down section to analyze specific accounts
3. Prioritize actions by savings potential
4. Execute immediate actions first (orphaned volumes, idle instances)
5. Re-run this report weekly/monthly to track progress

**To re-run with fresh data:**
```bash
# Step 1: Ensure SSO session is active
aws sso login --sso-session cost-analyzer-setup

# Step 2: Generate new multi-account cost report
uv run python multi_account_analyzer.py

# Step 3: Run all cells (Kernel -> Restart & Run All)
```